# TRADES DE plots and run

In [ ]:
import numpy as np
import os
import sys

In [ ]:
from pytrades import pytrades
from pytrades import ancillary as anc
from pytrades import constants as cst
from pytrades.de import DEEvolution

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib inline
anc.set_rcParams()

In [ ]:
exo_folder = os.path.abspath(
    "/path/to/explanet/system/folder"
)
run_folder = os.path.join(
    exo_folder,
    "main_simulation_folder",
    "run_pyde_subfolder"
)

Read `star.dat` file and get input stellar parameters

In [ ]:
# Mstar = 0.50 # 0.03          # Mstar [Msun]
# Rstar = 0.75 # 0.03          # Rstar [Rsun]
# print(Mstar, Rstar)
# or
# star file in run_folder as the first line of bodies.lst, i.e. star.dat
with open(os.path.join(run_folder, "bodies.lst"), "r") as bdf:
    star_file = bdf.readline().strip().split(" ")[0]
print(star_file)
with open(os.path.join(run_folder, star_file), "r") as sf:
    lines = sf.readlines()
Mstar = float(lines[0].strip().split(" ")[0])
Rstar = float(lines[1].strip().split(" ")[0])
print(Mstar, Rstar)

Load `PyDE` file and create object

In [ ]:
de = DEEvolution(run_folder)

You can convert masses and eccentricities from fitted parameters ($M_\textrm{p}/M_\star$, $\sqrt{e}\cos\omega$, $\sqrt{e}\sin\omega$)

In [ ]:
de.convert_masses(Mstar=Mstar, mass_unit='e') # mass_unit == 'e' for Earth, 'j' for Jupiter masses
de.get_eccentricity()
# de.convert_jitter()
# de.recenter_angles()

Simply print the best-fitted parameter obtained from `PyDE`

In [ ]:
de.de_par

Prepare plots output folder within `run_folder`

In [ ]:
plot_folder = os.path.join(
    run_folder,
    "plots"
)
os.makedirs(plot_folder, exist_ok=True)

Prepare labels, you can provides values to overplot, here defined as `true_pars` dictionary, set it to `None` if you do not want to overplot it

In [ ]:
true_pars = {k: None for k in de.par_names}

p = ["mass", "P", "ecc"]
npl = [2,3]
for pp in p:
    for nn in npl:
        k = "{}{}".format(pp, nn)
        true_pars[k] = None

In [ ]:
thinning = 100 # plot only every `thinning` iterations
perc_max = 50 # scale the fitness with lower value the `perc_max`-th percentile, e.g. `perc_max = 50` is the median

In [ ]:
for i_k, k in enumerate(true_pars.keys()):
    try:
        print(k)
        fig = de.evolution_plot_one_par(
            k,
            true_pars_dictionary=true_pars,
            thin_iter=100, 
            percentile_max=50
        )
        plt.show()
        fout = os.path.join(
            plot_folder,
            "00de_{:02d}_{}.png".format(i_k, k)
        )
        print(fout)
        fig.savefig(fout)
        plt.close(fig)
    except:
        print("Not present parameter {}".format(k))

Now we want to take the best-fit `PyDE` parameter and run `TRADES` and creates integration output files.  
Init the `TRADES` object, provides `run_folder`

In [ ]:
sim = pytrades.TRADESfolder(run_folder)

The `run_folder` has to have all the needed files by TRADES (see README file).  
Init `TRADES` parameters, constants, and data.

In [ ]:
sim.init_trades()

Let's define some info for the naming of the simulation and brief description:  
- `id_sim = 6` id numbering of the simulation, used for the unit of fortran file management, usually I would use 6 for `PyDE`  
- `sim_name = "de"` give a short simulation name
- `full_sim_name = "{:04d}_sim_{:s}".format(id_sim, sim_name)` full simulation name based on the `id_sim` and `sim_name`  
- `par_desc = "best-fit parameters from DE"` description of the simulation, it will appear in the `summary_parameters.dat` file

In [ ]:
id_sim = 6
sim_name = "de"
full_sim_name = "{:04d}_sim_{:s}".format(id_sim, sim_name)
out_folder = os.path.join(os.path.join(sim.full_path, full_sim_name), "")
os.makedirs(out_folder, exist_ok=True)

par_desc = "best-fit parameters from DE"

# convert the fitted parametrs into physical ones
_, de_phy, _, _ = anc.compute_physical_parameters(
    sim.n_bodies,
    de.de_par,
    sim.fitting_names,
    sim.system_parameters, # this is a vector used internally by TRADES and it is set by the files in the main folder
    mass_conv_factor=sim.m_factor,
    radius_conv_factor=sim.r_factor,
)

Run `TRADES` simulation and write output files (based on `configuration.yml` and `args.in` files).  
The function will return some statistics on the simulation, based on the parameter `de.de_par` provided:  

- `chi_square` $= \chi^{2}$ where the denominator, i.e. $\sigma^{2}$ is the sum in quadrature of the data error and of the RV jitters  
- `reduced_chi_square` $= \chi^{2}_\mathrm{r} = \chi^{2}/\mathrm{dof}$ reduced chi square with `dof = ndata - nfit` degrees of freedom  
- `lgllhd` $= \log \mathcal{L}$ log-likelihood  
- `lnprior` log-prior, if `priors.in` has no lines it will be 0.0  
- `ln_const` log-constant, depends on $2\pi$ and $\sigma$, already in the $\log \mathcal{L}$  
- `bic` $= -2\log\mathcal{L} + \mathrm{nfit}\times\log \mathrm{ndata}$ Bayesian Information Criterion  
- `check` parameter equal to `1/True` if simulation had no issue, else `0/False` for issues, such close encounters, out-of-boundaries, not physical parameters etc.  

The function can return nothing if `return_stats=False`.

In [ ]:
(
    chi_square,
    reduced_chi_square,
    lgllhd,
    lnprior,
    ln_const,
    bic,
    check,
) = sim.run_and_write_summary_files(
    full_sim_name,
    de.de_par,
    de_phy,
    id_sim=id_sim,
    sigma_fit=None,
    sigma_phy=None,
    par_description=par_desc,
    return_stats=True,
)

The previous cell run `TRADES` and creates a subfolder with files that we can read and plot outputs:  
Let's start with the Transit times: O-C diagram

In [ ]:
from pytrades import plot_oc as poc

Define the configuration object for the O-C plot

In [ ]:
oc_obj = poc.CLI_OC(
    full_path=os.path.join(sim.full_path, full_sim_name),
    idsim=6,
    lmflag=0, # keep it set to 0, needed for backward-compatibility with fortran binaries
    tscale=0.0, # value to subtract to x-axis, for the plot
    plot_title=False, # automatic title plot (True), or nothing (False) 
    ocunit='auto', # automatically determine best unit for the O-C y-axis, set "d", "m", "s" for days, minutes, and seconds respectively.
    samples_file=None, # here set it to None, you do not have a sample files generated from emcee now
    plot_samples= "ci", # not neede, no samples at this stage
    limits='obs', # scale limits to the observations obs or samples smp
    kep_ele=False, # default False, it creates a further plot of the keplerian elements for each T0
    linear_ephemeris=None, # input linear_ephemeris, here like a dict {2: {"Tref":[val, sigma], "Pref":[val, sigma]}}, default compute it internally
    legend="in", # where place legend, inside (in), outside (out), or no legend (none, not, as you want)
    idsource_name=None, # none, one color for everything, otherwise a dict {1: "Kepler", 2: "TESS", 3:"CHEOPS"}, numbers have to match source_id in input file
    color_map="nipy_spectral", # a cmap, a color, or dict {1: "C0", 2: "red", 3: [0.2, 0.4, 0, 1.0]}
)

Get the output T0s files  

In [ ]:
T0s_files = poc.get_simT0_file_list(oc_obj)

For each T0 file do the plot.  
By default it assumes that the planets are in this order:  

body_id | planet_name  
--------|------------
2       | b  
3       | c  

Provides a new `planet_name` for each `body_id`


In [ ]:
for body_id, f in T0s_files.items():
    print(f)
    fig = poc.plot_oc_T41(
        oc_obj,
        f,
        planet_name=None,
        samples=None,
        figsize=(5, 5),
        save_plot=True,
        show_plot=True,
    )
    plt.close(fig)